In [43]:
import subprocess
from threading import Thread
import time
import datetime
from twilio.rest import Client
from twilio_info import *
import logging 
import google.cloud.logging
import re


# Setup Logging
file_name = ("Ping Test")
logger = logging.getLogger(__name__)
logging.basicConfig(format="%(filename)s:%(lineno)s - %(funcName)s() %(message)s")
logger.setLevel(logging.INFO)

auth_file = "VirtualToolbox-5f6ecffdcf58.json"

client = google.cloud.logging.Client.from_service_account_json(auth_file)
client.setup_logging()


# Setup Twilio

client = Client(twilio_sid, twilio_auth_token)

def send_alert(msg):
    dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
    msg_body = "{}:  {}".format(dtnow, msg)
    try:
        logger.error(msg_body)
        message = client.messages.create(body=msg_body,from_=phone_num_from,to=phone_num_to)
        time.sleep(30)
        logger.info("Attemped to send alert text.  Status:  {}".format(message.status))
    except:
        logger.critical("Attemped to send alert text.  Status:  Failed")
 

# Setup Ping script

def ping_addr(ip_address):
    try:
        res = subprocess.call(['ping', '-c', '3', ip_address])
    except:
        res = 99
    dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
    if res == 0:
        print ("{} - ping to {} OK".format(dtnow, ip_address))
    elif res == 2:
        print ("{} - no response from {}".format(dtnow, ip_address))
    else:
        print ("{} - ping to {} failed!".format(dtnow, ip_address))
    return(res)
    

class address:
    def __init__(self, ip_address, mac_address):
        self.call_count = 0
        self.ip_address = ip_address
        self.mac_address = mac_address
        if not self.verify_address(ip_address, mac_address):
            self.ip_address = self.find_ip_address(mac_address)
    
    def find_mac_address(self, ip_address):
        ping_addr(ip_address)
        pid = subprocess.Popen(["arp", "-n", ip_address], stdout=subprocess.PIPE)
        s = pid.communicate()[0]
        mac = re.search(r"(([a-f\d]{1,2}\:){5}[a-f\d]{1,2})", str(s)).groups()[0]
        return(mac)

    def find_ip_address(self, mac_address):
        ping_addr('255.255.255.255')
        pid = subprocess.Popen(["arp", "-an"], stdout=subprocess.PIPE)
        s = pid.communicate()[0]
        ip_address = re.search(f"\(([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)\) at {mac_address.lower()}", str(s).lower())[1]
        return(ip_address)

    def verify_address(self, ip_address, mac_address):
        mac_address2 = self.find_mac_address(ip_address)
        return(mac_address.lower() == mac_address2.lower())
    
    def get_ip_address(self):
        self.call_count += 1
        if self.call_count > 10:
            self.__init__(self.ip_address, self.mac_address)
        return(self.ip_address)
    
def main(ip_address, mac_address):    
    addr = address(ip_address, mac_address)
    while True:
        alert_count = 0

        while alert_count < 4:
            status = ping_addr(addr.get_ip_address())
            count = 0

            while (status != 0)&(count<5):
                dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
                print("{} - Ping failed, backoff and retry".format(dtnow))
                time.sleep(3)
                status = ping_addr(ip_address)
                count += 1

            if status != 0:
                send_alert("Host {} is down!".format(ip_address))
                alert_count += 1

            time.sleep(30)

        if alert_count >= 4:
            dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
            print("{} - Exceeded alert count, backing off for 4 hours".format(dtnow))
            time.sleep(4*60*60)
            count = 0
            alert_count = 0

logger.info("Starting Ping Test")



ip_address = "10.0.2.217"
mac_address = '58:EF:68:EA:54:2D'

#main(ip_address, mac_address)


<ipython-input-43-0fbe08e17b6d>:118 - <module>() Starting Ping Test
Starting Ping Test
Starting Ping Test


In [2]:
main(ip_address, mac_address)

AttributeError: 'address' object has no attribute 'ping_addr'

In [35]:
def get_arp_table():
    ping_addr('255.255.255.255')
    pid = subprocess.Popen(["arp", "-an"], stdout=subprocess.PIPE)
    s = pid.communicate()[0]
    output = str(s).lower()
    output_list = []
    for output2 in output.split('?'):
        try:
            output_list.append(re.search("\(([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)\) at ([a-f0-9]{2}:[a-f0-9]{2}:[a-f0-9]{2}:[a-f0-9]{2}:[a-f0-9]{2}:[a-f0-9]{2})", output2).groups())
        except:
            pass
    return(output_list)

In [36]:
output_list = get_arp_table()

2022-08-15 08:11 - ping to 255.255.255.255 OK


In [37]:
output_list

[('10.0.2.1', 'd4:8c:b5:ac:98:b6'),
 ('10.0.2.14', 'dc:a6:32:4d:df:18'),
 ('10.0.2.157', 'ac:ae:19:a4:55:fd'),
 ('10.0.2.158', 'b8:27:eb:95:68:c3'),
 ('10.0.2.161', '30:fd:38:b0:22:da'),
 ('10.0.2.166', '2c:b4:3a:1e:1f:86'),
 ('10.0.2.182', 'd0:4d:2c:3a:8a:5d'),
 ('10.0.2.183', '2c:b4:3a:26:32:64'),
 ('10.0.2.186', 'fe:ba:99:ef:40:93'),
 ('10.0.2.187', '5c:52:1e:98:1c:d2'),
 ('10.0.2.188', 'aa:11:dc:74:8d:86'),
 ('10.0.2.204', '7c:bb:8a:f4:f2:b1'),
 ('10.0.2.214', '5c:52:30:9a:73:ed'),
 ('10.0.2.255', 'ff:ff:ff:ff:ff:ff'),
 ('10.0.2.255', 'ff:ff:ff:ff:ff:ff'),
 ('169.254.255.255', 'd4:8c:b5:ac:98:b6'),
 ('255.255.255.255', 'ff:ff:ff:ff:ff:ff')]

In [39]:
ping_addr('10.0.2.255')
get_arp_table()

2022-08-15 08:15 - ping to 10.0.2.255 OK
2022-08-15 08:15 - ping to 255.255.255.255 OK


[('10.0.2.0', 'ff:ff:ff:ff:ff:ff'),
 ('10.0.2.1', 'd4:8c:b5:ac:98:b6'),
 ('10.0.2.14', 'dc:a6:32:4d:df:18'),
 ('10.0.2.152', '58:ac:78:de:9c:40'),
 ('10.0.2.153', 'cc:7f:75:aa:47:40'),
 ('10.0.2.157', 'ac:ae:19:a4:55:fd'),
 ('10.0.2.158', 'b8:27:eb:95:68:c3'),
 ('10.0.2.161', '30:fd:38:b0:22:da'),
 ('10.0.2.166', '2c:b4:3a:1e:1f:86'),
 ('10.0.2.182', 'd0:4d:2c:3a:8a:5d'),
 ('10.0.2.183', '2c:b4:3a:26:32:64'),
 ('10.0.2.185', '44:61:32:46:c1:ff'),
 ('10.0.2.186', 'fe:ba:99:ef:40:93'),
 ('10.0.2.187', '5c:52:1e:98:1c:d2'),
 ('10.0.2.188', 'aa:11:dc:74:8d:86'),
 ('10.0.2.191', 'e4:ce:8f:67:67:e8'),
 ('10.0.2.196', '44:d2:44:64:8e:66'),
 ('10.0.2.204', '7c:bb:8a:f4:f2:b1'),
 ('10.0.2.206', '58:ef:68:ed:51:ed'),
 ('10.0.2.214', '5c:52:30:9a:73:ed'),
 ('10.0.2.217', '58:ef:68:ea:54:2d'),
 ('10.0.2.255', 'ff:ff:ff:ff:ff:ff'),
 ('169.254.255.255', 'd4:8c:b5:ac:98:b6'),
 ('255.255.255.255', 'ff:ff:ff:ff:ff:ff')]

CalledProcessError: Command '['ping', '-c1', '10.0.2.7']' returned non-zero exit status 2.

In [52]:
subnet = '10.0.2.'
for x in range(1, 255):
    ip_address = f"{subnet}{x}"
    print(ip_address)
    subprocess.call(['ping', '-c', '1', ip_address])
    

10.0.2.1
10.0.2.2
10.0.2.3
10.0.2.4
10.0.2.5
10.0.2.6
10.0.2.7


KeyboardInterrupt: 

In [59]:
subprocess.call(['nmap', '-sp', '-n', '10.0.2.0/24'])

KeyboardInterrupt: 

In [ ]:
get_arp_table()

In [22]:
re.match("\(([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)\) at ([a-f0-9]{2}:[a-f0-9]{2}:[a-f0-9]{2}:[a-f0-9]{2}):[a-f0-9]{2}:[a-f0-9]{2}", output)

["b'",
 ' (10.0.2.1) at d4:8c:b5:ac:98:b6 on en0 ifscope [ethernet]\\n',
 ' (10.0.2.11) at 0:0:5e:0:1:1 on en0 ifscope [ethernet]\\n',
 ' (10.0.2.14) at dc:a6:32:4d:df:18 on en0 ifscope [ethernet]\\n',
 ' (10.0.2.154) at 3c:7:54:45:bb:c2 on en0 ifscope permanent [ethernet]\\n',
 ' (10.0.2.157) at ac:ae:19:a4:55:fd on en0 ifscope [ethernet]\\n',
 ' (10.0.2.158) at b8:27:eb:95:68:c3 on en0 ifscope [ethernet]\\n',
 ' (10.0.2.159) at 8:5:81:2a:e6:12 on en0 ifscope [ethernet]\\n',
 ' (10.0.2.161) at 30:fd:38:b0:22:da on en0 ifscope [ethernet]\\n',
 ' (10.0.2.166) at 2c:b4:3a:1e:1f:86 on en0 ifscope [ethernet]\\n',
 ' (10.0.2.168) at 0:17:88:68:46:a4 on en0 ifscope [ethernet]\\n',
 ' (10.0.2.170) at 44:33:4c:b3:27:f on en0 ifscope [ethernet]\\n',
 ' (10.0.2.173) at bc:83:85:54:d:7 on en0 ifscope [ethernet]\\n',
 ' (10.0.2.182) at d0:4d:2c:3a:8a:5d on en0 ifscope [ethernet]\\n',
 ' (10.0.2.183) at 2c:b4:3a:26:32:64 on en0 ifscope [ethernet]\\n',
 ' (10.0.2.186) at fe:ba:99:ef:40:93 on en0 ifs

In [14]:
re.search("\(([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)\) at ([a-f0-9]{2}:)", output)

<re.Match object; span=(4, 21), match='(10.0.2.1) at d4:'>

In [9]:
addr = address(ip_address, mac_address)

2022-08-15 07:28 - ping to 10.0.2.217 OK
